In [1]:
from pathlib import Path

from pydantic import BaseModel

from pyalab import AspirateParameters
from pyalab import Deck
from pyalab import DeckLayout
from pyalab import DeckPosition
from pyalab import DispenseParameters
from pyalab import LabwareOrientation
from pyalab import MultiDispense
from pyalab import Pipette
from pyalab import PipettingLocation
from pyalab import Plate
from pyalab import Program
from pyalab import Reservoir
from pyalab import SetInitialVolume
from pyalab import StandardDeckNames
from pyalab import Tip
from pyalab import TipChangeMode

In [2]:
# Define your labware
reservoir = Reservoir(name="INTEGRA 25 ml Multichannel Reservoir", display_name="Equilibration")
# DWP Col 1: Sample
# DWP Col 2-4: Washes 1-3
# DWP Col 5-6: Elutions 1-2
# DWP Col 8: Neutralization Buffer

dwp = Plate(name="INTEGRA 96 Deepwell V-Bottom Plate", display_name="DWP")

In [3]:
# Initialize variables you'll use in your Program
debug_mode = True


class ImcsStepParams(BaseModel):
    mix_volume: float
    aspirate_delay: int
    aspirate_speed: int
    dispense_delay: int
    dispense_speed: int
    mix_count: int
    initial_well_volume: float
    start_height: float


equilibration_params = ImcsStepParams(
    mix_volume=800,
    aspirate_delay=9 if not debug_mode else 0,
    dispense_delay=20 if not debug_mode else 0,
    mix_count=3 if not debug_mode else 2,
    initial_well_volume=20000,
    aspirate_speed=7,
    dispense_speed=8,
    start_height=4.5,
)
dwp_plate_column_index = 0
type ColumnIndex = int
dwp_actions: dict[ColumnIndex, ImcsStepParams] = {}
# sample_params
dwp_actions[dwp_plate_column_index] = ImcsStepParams(
    mix_volume=800,
    aspirate_delay=7 if not debug_mode else 0,
    dispense_delay=20 if not debug_mode else 0,
    mix_count=30 if not debug_mode else 2,
    initial_well_volume=1000,
    aspirate_speed=7,
    dispense_speed=8,
    start_height=4.5,
)
dwp_plate_column_index += 1
# washes
dwp_actions[dwp_plate_column_index] = ImcsStepParams(
    mix_volume=850,
    aspirate_delay=7 if not debug_mode else 0,
    dispense_delay=20  # 20 seconds seemed to work better for this, see notes in experiment EXP992182
    if not debug_mode
    else 0,
    mix_count=3 if not debug_mode else 2,
    initial_well_volume=1000,
    aspirate_speed=7,
    dispense_speed=8,
    start_height=4.5,
)
dwp_plate_column_index += 1
dwp_actions[dwp_plate_column_index] = ImcsStepParams(
    mix_volume=850,
    aspirate_delay=7 if not debug_mode else 0,
    dispense_delay=20 if not debug_mode else 0,
    mix_count=2 if not debug_mode else 1,
    initial_well_volume=1000,
    aspirate_speed=7,
    dispense_speed=8,
    start_height=4.5,
)
dwp_plate_column_index += 1
dwp_actions[dwp_plate_column_index] = ImcsStepParams(
    mix_volume=850,
    aspirate_delay=7 if not debug_mode else 0,
    dispense_delay=20 if not debug_mode else 0,
    mix_count=2 if not debug_mode else 1,
    initial_well_volume=1000,
    aspirate_speed=7,
    dispense_speed=8,
    start_height=4.5,
)
dwp_plate_column_index += 1
# elutions
elution_columns: list[ColumnIndex] = []
dwp_actions[dwp_plate_column_index] = ImcsStepParams(
    mix_volume=125,
    aspirate_delay=5 if not debug_mode else 0,
    dispense_delay=10 if not debug_mode else 0,
    mix_count=5 if not debug_mode else 2,
    initial_well_volume=150,
    aspirate_speed=7,
    dispense_speed=8,
    start_height=4.5,
)
elution_columns.append(dwp_plate_column_index)
dwp_plate_column_index += 1
dwp_actions[dwp_plate_column_index] = ImcsStepParams(
    mix_volume=125,
    aspirate_delay=5 if not debug_mode else 0,
    dispense_delay=10 if not debug_mode else 0,
    mix_count=5 if not debug_mode else 2,
    initial_well_volume=150,
    aspirate_speed=7,
    dispense_speed=8,
    start_height=4.5,
)
elution_columns.append(dwp_plate_column_index)
dwp_plate_column_index += 1

# neutralizations
neutralization_buffer_column_index = 7
neutralization_source_volume = 500
neutralization_volume = 100

In [4]:
# Instantiate the Program
program = Program(
    deck_layouts=[
        DeckLayout(
            deck=Deck(name=StandardDeckNames.THREE_POSITION.value),
            labware={
                DeckPosition(name="A", orientation=LabwareOrientation.A1_NW_CORNER): reservoir,
                DeckPosition(name="B", orientation=LabwareOrientation.A1_NE_CORNER): dwp,
            },
        )
    ],
    display_name="imcs-demo",
    description="IMCS Protocol",
    pipette=Pipette(name="VIAFLO TWELVE 1250 µl"),
    tip=Tip(name="1250 µl GripTip Sterile Filter Low retention"),
)
reservoir_section_index = program.get_section_index_for_labware(reservoir)
dwp_section_index = program.get_section_index_for_labware(dwp)

In [5]:
# Add steps to the Program

In [6]:
# Step for defining initial volumes
program.add_step(
    SetInitialVolume(
        labware=reservoir,
        section_index=reservoir_section_index,
        column_index=0,
        volume=equilibration_params.initial_well_volume,
        pipette_span=9,
    )
)
for column_index, params in dwp_actions.items():
    program.add_step(
        SetInitialVolume(
            labware=dwp,
            section_index=dwp_section_index,
            column_index=column_index,
            volume=params.initial_well_volume,
        )
    )
program.add_step(
    SetInitialVolume(
        labware=dwp,
        section_index=dwp_section_index,
        column_index=neutralization_buffer_column_index,
        volume=neutralization_source_volume,
    )
)

In [7]:
# Equilibrate the tips
reservoir_location = PipettingLocation(
    labware=reservoir, deck_section_index=reservoir_section_index, column_index=0, upper_left_row_index=0
)
for _ in range(equilibration_params.mix_count):
    program.add_step(
        MultiDispense(
            source=reservoir_location,
            destinations=[(reservoir_location, equilibration_params.mix_volume)],
            aspirate_parameters=AspirateParameters(
                liquid_speed=equilibration_params.aspirate_speed, post_delay=equilibration_params.aspirate_delay
            ),
            dispense_parameters=DispenseParameters(
                liquid_speed=equilibration_params.dispense_speed, post_delay=equilibration_params.dispense_delay
            ),
            tip_change_mode=TipChangeMode.NO_CHANGE,
            pipette_span=9,
        )
    )
# Steps for purification
for process_index, (column_index, params) in enumerate(dwp_actions.items()):
    dwp_column_location = PipettingLocation(
        labware=dwp, deck_section_index=dwp_section_index, column_index=column_index, upper_left_row_index=0
    )
    for mix_count in range(params.mix_count):
        is_last_step = process_index == len(dwp_actions.keys()) - 1 and mix_count == params.mix_count - 1
        program.add_step(
            MultiDispense(
                source=dwp_column_location,
                destinations=[(dwp_column_location, params.mix_volume)],
                aspirate_parameters=AspirateParameters(
                    liquid_speed=params.aspirate_speed, post_delay=params.aspirate_delay
                ),
                dispense_parameters=DispenseParameters(
                    liquid_speed=params.dispense_speed, post_delay=params.dispense_delay
                ),
                tip_change_mode=TipChangeMode.NO_CHANGE if not is_last_step else TipChangeMode.AFTER_STEP,
            )
        )

# Steps for Neutralization
source_column_location = PipettingLocation(
    labware=dwp,
    deck_section_index=dwp_section_index,
    column_index=neutralization_buffer_column_index,
    upper_left_row_index=0,
)
for elution_column_index in elution_columns:
    elution_column_location = PipettingLocation(
        labware=dwp, deck_section_index=dwp_section_index, column_index=elution_column_index, upper_left_row_index=0
    )
    program.add_step(
        MultiDispense(
            source=source_column_location,
            destinations=[(elution_column_location, neutralization_volume)],
        )
    )

In [8]:
# Save the program file.
program.dump_xml(Path("/home") / "jovyan" / "docs" / "imcs-demo.iaa")

In [9]:
# Great! You should now see the file over in the left "file explorer" pane, and can right-click on it and select Download to save it to your local machine.